# Quantum Convolutional Neural Network

This is an implementation of 8 Qubit QCNN circuit (Quantum Convolutional Neural Networks by Iris Cong et al, 2019) and Tree Tensor Network circuit (Hierarchical Quantum Classifier by Ed Grant et al, 2018) with the comparisons among various ansatze.

In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)

# QCNN Circuit

Here are various unitary ansatze to test

In [2]:
# Unitraies for Convolutional Layers 
def U_TTN(params, wires): # 2 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_5(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRZ(params[4], wires = [wires[1], wires[0]])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])
    
def U_6(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRX(params[4], wires = [wires[1], wires[0]])
    qml.CRX(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])

def U_9(params, wires): # 2 params
    qml.Hadamard(wires = wires[0])
    qml.Hadamard(wires = wires[1])
    qml.CZ(wires = [wires[0],wires[1]])
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])

def U_13(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRZ(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])

def U_14(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRX(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRX(params[5], wires = [wires[0], wires[1]])

def U_15(params, wires): # 4 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[1], wires[0]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_SO4(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])
    qml.RY(params[4], wires = wires[0])
    qml.RY(params[5], wires = wires[1])

# Unitraies for Pooling and Fully Connected Layers
def V_0(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

def V_1(theta, wires):
    qml.CRX(theta, wires = [wires[0], wires[1]])

def F(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

Below is general circuit structures used in Quantum Convolutional Neural Network (QCNN) circuits.

In [3]:
# Convolution Layer1
def conv_layer1(U, params):
    U(params, wires = [0,7])
    for i in range (0,8,2):
        U(params, wires = [i, i+1])
    for i in range (1,7,2):
        U(params, wires = [i, i+1])
    
def conv_layer2(U, params):
    U(params, wires = [0,2])
    U(params, wires = [4,6])
    U(params, wires = [2,4])
    U(params, wires = [0,6])
    
def pooling_layer1(V_0, V_1, params):
    for i in range(0,8,2):
        V_0(params[0], wires = [i+1, i])
    for i in range(0,8,2):
        qml.PauliX(wires = i+1)
    for i in range(0,8,2):
        V_1(params[1], wires = [i+1, i])
        

def pooling_layer2(V_0, V_1, params): # 2params
    V_0(params[0], wires = [2,0])
    V_0(params[0], wires = [6,4])
    
    qml.PauliX(wires = 2)
    qml.PauliX(wires = 6)
    
    V_1(params[1], wires = [2,0])
    V_1(params[1], wires = [6,4])

Here we define various possible embedding methods.

In [15]:
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding

def data_embedding(X, embedding_type = 'Amplitude'):
    if embedding_type == 'Amplitude':
        AmplitudeEmbedding(X, wires = range(8), normalize = True)
    elif embedding_type == 'Angle':
        AngleEmbedding(X, wires = range(8), rotation = 'Y')
    elif embedding_type == 'Hybrid':
        X1 = X[:2**4]
        X2 = X[2**4:2**5]
        
        AmplitudeEmbedding(X1, wires = range(4), normalize = True)
        AmplitudeEmbedding(X2, wires = range(4,8), normalize = True)

Define QCNN circuit with given Unitary Ansatz and embedding method.

In [5]:
dev = qml.device('default.qubit', wires = 8)

@qml.qnode(dev)
def QCNN(X, params, U, U_params, embedding_type = 'Amplitude'):

    param1 = params[0:U_params]
    param2 = params[U_params:U_params + 2]
    param3 = params[U_params + 2: 2*U_params + 2]
    param4 = params[2*U_params + 2: 2*U_params + 4]
    param5 = params[2*U_params + 4]
    
    # Data Embedding
    data_embedding(X, embedding_type = embedding_type)
    
    #Quantum Convolutional Neural Network
    if U == 'U_TTN':
        conv_layer1(U_TTN, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_TTN, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_5':
        conv_layer1(U_5, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_5, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_6':
        conv_layer1(U_6, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_6, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_9':
        conv_layer1(U_9, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_9, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_13':
        conv_layer1(U_13, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_13, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_14':
        conv_layer1(U_14, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_14, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_15':
        conv_layer1(U_15, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_15, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
    
    elif U == 'U_SO4':
        conv_layer1(U_SO4, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_SO4, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    return qml.expval(qml.PauliZ(4))

Below is Hierarchical Quantum Classifier structure with different Ansatze

In [12]:
dev_TTN = qml.device('default.qubit', wires = 8)

@qml.qnode(dev_TTN)
def Hierarchical_classifier(X, params, U, U_params, embedding_type = 'Amplitude'):
    
    param1 = params[0 * U_params:1 * U_params]
    param2 = params[1 * U_params:2 * U_params]
    param3 = params[2 * U_params:3 * U_params]
    param4 = params[3 * U_params:4 * U_params]
    param5 = params[4 * U_params:5 * U_params]
    param6 = params[5 * U_params:6 * U_params]
    param7 = params[6 * U_params:7 * U_params]
    
    data_embedding(X, embedding_type = embedding_type) 
    if U == 'U_TTN':
        # layer 1
        U_TTN(param1, wires = [0,1])
        U_TTN(param2, wires = [2,3])
        U_TTN(param3, wires = [4,5])
        U_TTN(param4, wires = [6,7])
        # layer 2
        U_TTN(param5, wires = [1,3])
        U_TTN(param6, wires = [5,7])
        # layer 3
        U_TTN(param7, wires = [3,7])
    elif U == 'U_5':
        # layer 1
        U_5(param1, wires = [0,1])
        U_5(param2, wires = [2,3])
        U_5(param3, wires = [4,5])
        U_5(param4, wires = [6,7])
        # layer 2
        U_5(param5, wires = [1,3])
        U_5(param6, wires = [5,7])
        # layer 3
        U_5(param7, wires = [3,7])
    elif U == 'U_6':
        # layer 1
        U_6(param1, wires = [0,1])
        U_6(param2, wires = [2,3])
        U_6(param3, wires = [4,5])
        U_6(param4, wires = [6,7])
        # layer 2
        U_6(param5, wires = [1,3])
        U_6(param6, wires = [5,7])
        # layer 3
        U_6(param7, wires = [3,7])
    elif U == 'U_13':
        # layer 1
        U_13(param1, wires = [0,1])
        U_13(param2, wires = [2,3])
        U_13(param3, wires = [4,5])
        U_13(param4, wires = [6,7])
        # layer 2
        U_13(param5, wires = [1,3])
        U_13(param6, wires = [5,7])
        # layer 3
        U_13(param7, wires = [3,7])
    elif U == 'U_14':
        # layer 1
        U_14(param1, wires = [0,1])
        U_14(param2, wires = [2,3])
        U_14(param3, wires = [4,5])
        U_14(param4, wires = [6,7])
        # layer 2
        U_14(param5, wires = [1,3])
        U_14(param6, wires = [5,7])
        # layer 3
        U_14(param7, wires = [3,7])
    elif U == 'U_15':
        # layer 1
        U_15(param1, wires = [0,1])
        U_15(param2, wires = [2,3])
        U_15(param3, wires = [4,5])
        U_15(param4, wires = [6,7])
        # layer 2
        U_15(param5, wires = [1,3])
        U_15(param6, wires = [5,7])
        # layer 3
        U_15(param7, wires = [3,7])
    elif U == 'U_SO4':
        # layer 1
        U_SO4(param1, wires = [0,1])
        U_SO4(param2, wires = [2,3])
        U_SO4(param3, wires = [4,5])
        U_SO4(param4, wires = [6,7])
        # layer 2
        U_SO4(param5, wires = [1,3])
        U_SO4(param6, wires = [5,7])
        # layer 3
        U_SO4(param7, wires = [3,7])
    

    return qml.expval(qml.PauliZ(7))

# Training Quantum Circuits

In [7]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(params, X, Y, U, U_params, embedding_type, circuit):
    if circuit == 'QCNN':
        predictions = [QCNN(x, params, U, U_params, embedding_type) for x in X]
    elif circuit == 'Hierarchical':
        predictions = [Hierarchical_classifier(x, params, U, U_params, embedding_type) for x in X]
    
    return square_loss(Y, predictions)

def accuracy_test_binary(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)

def accuracy_test_one_class(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 0.5:
            acc = acc + 1
    return acc / len(labels)

In [8]:
def circuit_training(X_train, Y_train, U, U_params, embedding_type, circuit):
    if circuit == 'QCNN':
        total_params = U_params * 2 + 2 * 2 + 1
    elif circuit == 'Hierarchical':
        total_params = U_params * 7
        
    params = np.random.randn(total_params)
    steps = 150
    learning_rate = 0.1
    batch_size =25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, embedding_type, circuit), params)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return params

### MNIST Data loading and processing

Use PCA and Autoencoder to reduce it into 8 features. We test both one-class classification (labeling 0 and 1) and binary classification (labeling -1 and 1).

In [19]:
import tensorflow as tf
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

def data_load_and_process(classes = [0,1], feature_reduction = 'resize256', binary = True):
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0 #normalize the data
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    x_train_01, x_test_01 = x_train[x_train_filter_01], x_test[x_test_filter_01]
    y_train_01, y_test_01 = y_train[x_train_filter_01], y_test[x_test_filter_01]
    
    if binary == False:
        y_train_01 = [1 if y ==classes[0] else 0 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else 0 for y in y_test_01]
    elif binary == True:
        y_train_01 = [1 if y ==classes[0] else -1 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else -1 for y in y_test_01]
        
    
    if feature_reduction == 'resize256':   
        x_train_01 = tf.image.resize(x_train_01[:], (256, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (256, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01) 
        return x_train_01, x_test_01, y_train_01, y_test_01
    
    elif feature_reduction == 'pca8' or feature_reduction == 'pca32':
        x_train_01 = tf.image.resize(x_train_01[:], (784, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (784, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01)
        
        if feature_reduction == 'pca8':
            pca = PCA(8)
        elif feature_reduction == 'pca32':
            pca = PCA(32)
            
        x_train_01 = pca.fit_transform(x_train_01)
        x_test_01 = pca.transform(x_test_01)
        
        #Rescale for angle embedding
        if feature_reduction == 'pca8':
            x_train_01, x_test_01 = (x_train_01 + 10) * (np.pi / 20), (x_test_01 + 10) * (np.pi / 20)
        
        return x_train_01, x_test_01, y_train_01, y_test_01

    elif feature_reduction == 'autoencoder8' or feature_reduction == 'autoencoder32':
        if feature_reduction == 'autoencoder8':
            latent_dim = 8 
        elif feature_reduction == 'autoencoder32':
            latent_dim = 32
            
        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim   
                self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                  layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                layers.Dense(784, activation='sigmoid'),
                layers.Reshape((28, 28))
                ])
            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded
        
        autoencoder = Autoencoder(latent_dim)
        
        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(x_train_01, x_train_01,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_01, x_test_01))
        
        x_train_01, x_test_01 = autoencoder.encoder(x_train_01).numpy(), autoencoder.encoder(x_test_01).numpy()
        #Rescale for Angle Embedding
        if feature_reduction == 'autoencoder8':
            x_train_01, x_test_01 = x_train_01 * (np.pi / 50), x_test_01 * (np.pi / 50)
        
        return x_train_01, x_test_01, y_train_01, y_test_01

# Results

Benchmarking function trains and performs accuracy tests for all the given unitary ansatze, feature reduction methods, and circuit structures.

In [31]:
def Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True):
    I = len(Unitaries)
    J = len(Encodings)
    All_predictions = []
    
    for i in range(I):
        for j in range(J):
            U = Unitaries[i]
            U_params = U_num_params[i]
            Encoding = Encodings[j]
            if Encoding == 'resize256':
                Embedding = 'Amplitude'
                X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'resize256', binary = binary)
            elif Encoding == 'pca8':
                Embedding = 'Angle'
                X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'pca8', binary = binary)
            elif Encoding == 'autoencoder8':
                Embedding = 'Angle'
                X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'autoencoder8', binary = binary)
            elif Encoding == 'pca32':
                Embedding = 'Hybird'
                X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'pca32', binary = binary)
                print("Checkpoint!")
            elif Encoding == 'autoencoder32':
                Embedding = 'Hybrid'
                X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'autoencoder32', binary = binary)
            print("\n")
            print("Loss History for " + circuit + " circuits, "+ U + " " + Encoding)
            trained_params = circuit_training(X_train, Y_train, U, U_params, Embedding, circuit)
            
            if circuit == 'QCNN':
                predictions = [QCNN(x, trained_params, U, U_params, Embedding) for x in X_test]
            elif circuit == 'Hierarchical':
                predictions = [Hierarchical_classifier(x, trained_params, U, U_params, Embedding) for x in X_test]
                
                
            if binary == True:
                accuracy = accuracy_test_binary(predictions, Y_test)
            elif binary == False:
                accuracy = accuracy_test_one_class(predictions, Y_test)
                
            print("Accuracy for " + U + " " + Encoding + " :" + str(accuracy))

### 1. Results for QCNN circuit

#### 1.1 Binary Classification with 1, -1 labels

In [ ]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'QCNN'

In [ ]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

#### 1.2 One Class Classification with labels 0,1

In [ ]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4']
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'QCNN'

In [ ]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = False)

## 2. Results for Hierarchical Classifier circuit

#### 2.1 Binary Classification with 1, -1 labeling

In [ ]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'Hierarchical'

In [ ]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

#### 2.2 One Class Classification with 0, 1 labeling

In [ ]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'Hierarchical'

In [ ]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = False)

## 3. Results for Hybrid Embedding Method

In [29]:
Unitariies = ['U_13']
U_num_params = [6]
Encodings = ['pca32', 'autoencoder32']
classes = [0,1]
circuit = 'QCNN'

In [30]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

Checkpoint!


Loss History for QCNN circuits, U_TTN pca32
iteration:  0  cost:  1.0007882901912695
iteration:  10  cost:  0.9975939198755492
iteration:  20  cost:  0.99729377231524
iteration:  30  cost:  0.9848844712268882
iteration:  40  cost:  0.9733605132211216
iteration:  50  cost:  1.053139224271328
iteration:  60  cost:  1.0067457495126566
iteration:  70  cost:  0.9984799356222173
iteration:  80  cost:  1.0200157500378173
iteration:  90  cost:  1.0412949532645692
iteration:  100  cost:  0.9984898971228858
iteration:  110  cost:  0.990628141335675
iteration:  120  cost:  0.9843699769365624
iteration:  130  cost:  0.9984205115244659
iteration:  140  cost:  0.9814022489992137
Accuracy for U_TTN pca32 :0.5366430260047281


Loss History for QCNN circuits, U_TTN autoencoder32
iteration:  0  cost:  1.0116531643166582
iteration:  10  cost:  1.0059244624322021
iteration:  20  cost:  1.0124025805176873
iteration:  30  cost:  0.9946292649195343
iteration:  40  cost:  0.9856611142363283
iter

IndexError: list index out of range

In [32]:
Unitariies = ['U_TTN', 'U_5', 'U_13']
U_num_params = [2, 10, 6]
Encodings = ['pca32', 'autoencoder32']
classes = [0,1]
circuit = 'Hierarchical'

In [33]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

Checkpoint!


Loss History for Hierarchical circuits, U_TTN pca32
iteration:  0  cost:  0.9604389162036482
iteration:  10  cost:  1.008092047574757
iteration:  20  cost:  0.998832777454756
iteration:  30  cost:  1.0005264789507329
iteration:  40  cost:  0.9999745116852204
iteration:  50  cost:  0.9998569756973598
iteration:  60  cost:  0.9998090659720563
iteration:  70  cost:  0.9931545484285103
iteration:  80  cost:  0.9825914774392033
iteration:  90  cost:  0.9856004716914416
iteration:  100  cost:  1.0075204635399833
iteration:  110  cost:  0.9997885814100224
iteration:  120  cost:  0.9975318046747711
iteration:  130  cost:  0.9886045716401687
iteration:  140  cost:  1.0066788917566445
Accuracy for U_TTN pca32 :0.5366430260047281
Train on 12665 samples, validate on 2115 samples
Epoch 1/10
12665/12665 [==============================] - 1s 79us/sample - loss: 0.0452 - val_loss: 0.0244
Epoch 2/10
12665/12665 [==============================] - 1s 52us/sample - loss: 0.0206 - val_loss: 0


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_TTN autoencoder32


DeviceError: Operation QubitStateVector cannot be used after other Operations have already been applied on a default.qubit.autograd device.